In [1]:
from local_tree_decomp import *

# from local_helpers import *
from help_functions import *

# from local_hom_count import *
# from local_hom_count_int import *
from local_hom_count_best import *
# from local_hom_count_int_dict import *

# from hom_count_bounded_degree import *


In [2]:
graph = graphs.CompleteBipartiteGraph(1, 2)
target_graph = graphs.CompleteGraph(4)

graph_clr = [1, 0, 1]
target_clr = [1, 1, 1, 0]

count_homomorphisms_best(graph, target_graph, graph_clr, target_clr, colourful=True)

6

In [16]:
graph = graphs.CompleteBipartiteGraph(1, 3)
random_graph = graphs.RandomGNP(1, 0.55, seed=42)

%prun count_homomorphisms_best(graph, random_graph)

         3635 function calls (3564 primitive calls) in 0.006 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       24    0.001    0.000    0.001    0.000 set.py:472(__init__)
       59    0.000    0.000    0.000    0.000 {built-in method builtins.hasattr}
      393    0.000    0.000    0.001    0.000 set.py:1046(__hash__)
      393    0.000    0.000    0.000    0.000 set.py:1018(frozenset)
        1    0.000    0.000    0.003    0.003 local_tree_decomp.py:5(make_nice_tree_decomposition)
        1    0.000    0.000    0.006    0.006 local_hom_count_best.py:10(count_homomorphisms_best)
      545    0.000    0.000    0.000    0.000 set.py:767(object)
      152    0.000    0.000    0.000    0.000 set.py:995(set)
       28    0.000    0.000    0.000    0.000 digraph.py:1223(neighbor_out_iterator)
        5    0.000    0.000    0.000    0.000 generic_graph.py:11768(add_edges)
      3/2    0.000    0.000    0.001    0.000 graph.p

In [ ]:
count_homomorphisms_best(graph, random_graph)

In [9]:
1.0 * random_graph.density()

0.803983935742972

In [6]:
%load_ext line_profiler

from local_hom_count_best import count_homomorphisms_best

%lprun -f count_homomorphisms_best(graph, target_graph)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


UsageError: Could not find module count_homomorphisms_best(graph,.
SyntaxError: '(' was never closed (<string>, line 1)


In [ ]:
def speed_test(graph, target_graph, naive=True):
    basic_result = count_homomorphisms(graph, target_graph)
    int_rep_result = count_homomorphisms_int(graph, target_graph)
    better_int_rep_result = count_homomorphisms_int_pre(graph, target_graph)
    dict_int_rep_result = count_homomorphisms_int_dict(graph, target_graph)

    results = [basic_result, int_rep_result, better_int_rep_result, dict_int_rep_result]

    if naive:
        brute_force_result = len(enumerate_homomorphisms(graph, target_graph))
        results.append(brute_force_result)

    if all(result == better_int_rep_result for result in results):
        print('result:', better_int_rep_result)

        if naive:
            print('\nBrute force:')
            %timeit len(enumerate_homomorphisms(graph, target_graph))
        
        print('\nBasic:')
        %timeit count_homomorphisms(graph, target_graph)
        
        print('\nInt representation:')
        %timeit count_homomorphisms_int(graph, target_graph)
    
        print('\nBetter int representation:')
        %timeit count_homomorphisms_int_pre(graph, target_graph)

        print('\nDict int representation:')
        %timeit count_homomorphisms_int_dict(graph, target_graph)


In [ ]:
g = graphs.CycleGraph(4)

tree_decomp = g.treewidth(certificate=True)
nice_tree_decomp = make_nice_tree_decomposition(g, tree_decomp)
root = sorted(nice_tree_decomp)[0]
dir_labelled_TD = label_nice_tree_decomposition(nice_tree_decomp, root, directed=True)

In [ ]:
g

In [ ]:
for i in dir_labelled_TD.vertices():
    print(i, dir_labelled_TD.get_vertex(i))

In [ ]:
dir_labelled_TD.edges(labels=False)

In [ ]:
dir_labelled_TD

In [ ]:
dir_labelled_TD.vertices()

In [ ]:
node_changes(dir_labelled_TD)

In [ ]:
from local_hom_count_colour import *

graph = graphs.CompleteBipartiteGraph(1, 2)
target_graph = graphs.CompleteGraph(3)

graph_clr = [0, 1, 2]
target_clr = [0, 1, 2]

count_homomorphisms_best_colour(graph, target_graph, graph_clr, target_clr)

In [ ]:
graph

In [ ]:
target_graph

In [ ]:
tree_decomp = graph.treewidth(certificate=True)
nice_tree_decomp = make_nice_tree_decomposition(graph, tree_decomp)
root = sorted(nice_tree_decomp)[0]
dir_labelled_TD = label_nice_tree_decomposition(nice_tree_decomp, root, directed=True)

In [ ]:
dir_labelled_TD

In [14]:
def best_speed_test(graph, target_graph, graph_clr, target_clr):
    # best_result = count_homomorphisms_best(graph, target_graph)
    # best_colour_result = count_homomorphisms_best_colour(graph, target_graph, graph_clr, target_clr)

    print('Best result:')
    %timeit count_homomorphisms_best(graph, target_graph)

    print('\nBest colour result:')
    %timeit count_homomorphisms_best(graph, target_graph, graph_clr, target_clr, colourful=True)


In [15]:
import random
random.seed(int(42))

graph = graphs.CompleteBipartiteGraph(1, 2)
# target_graph = graphs.CompleteGraph(3)

graph_clr = [0, 1, 1]
# target_clr = [0, 1, 2]

for i in range(1, 11):
    N = 100
    density = 0.1 * i
    # complete_graph = graphs.CompleteGraph(N)
    random_graph = graphs.RandomGNP(N, density, seed=42)
    random_clr = [random.randint(0, 1) for _ in range(N)]
    print('density: ', density)
    best_speed_test(graph, random_graph, graph_clr, random_clr)
    print()

density:  0.100000000000000
Best result:
75.4 ms ± 3.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Best colour result:
484 ms ± 19.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

density:  0.200000000000000
Best result:
76.6 ms ± 4.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Best colour result:
533 ms ± 31.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

density:  0.300000000000000
Best result:
75.1 ms ± 3.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Best colour result:
552 ms ± 51.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

density:  0.400000000000000
Best result:
84.2 ms ± 15.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Best colour result:
554 ms ± 45.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

density:  0.500000000000000
Best result:
93.2 ms ± 6.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Best colour result:
569 ms ± 60.7 ms per loop (mean ± std. dev. of 7 runs, 1 lo

In [ ]:
import random

graph = graphs.CompleteBipartiteGraph(1, 3)
graph_clr = [0, 1, 1, 1]

target_graph = graphs.PetersenGraph()

max_num = 0

for _ in range(500):
    random_clr = [random.randint(0, 1) for _ in range(10)]
    num = count_homomorphisms_best(graph, target_graph, graph_clr, random_clr, colourful=True)
    if num > max_num:
        max_num = num

print(max_num)

In [ ]:
count_homomorphisms_best(graph, target_graph)